# PSKG Simple Neo4j Query Example

The Patient Safety Knowledge Graph (PSKG) is implemented on a graph database called Neo4J, and runs in aibench.  The ZScalar application (available in the AZ Software Store) enables access to the PSKG directly through a browser interface:

* [http://kckb075.1000-672.service.azaibenchdev.net:7474/browser/](http://kckb075.1000-672.service.azaibenchdev.net:7474/browser/) (TEST instance)
* [http://pskg.1000-672.service.azaibenchdev.net:7474/browser/](http://kckb075.1000-672.service.azaibenchdev.net:7474/browser/) (PRODUCTION instance)

as well as programatically from from Python.  This notebook is a self contained example to connect to the database, run a query, and return results.

**Important**: This notebook requires the neo4j package (install using pip) as well as pandas.

In [ ]:
%load_ext lab_black

In [ ]:
import pandas as pd

# Library for accessing neo4j
from neo4j import GraphDatabase

In [ ]:
NEO4J_USER = "neo4j"
NEO4J_PW = "pskg"
NEO4J_URI = "neo4j://pskg.1000-672.service.azaibenchdev.net:7687"

In [ ]:
# this function (and other helpers for working with PSKG are available)
def run_query(tx, query, verbose=False):
    """
    Minimal query run, returns results as a dataframe

    Parameters
    ----------
    tx: Neo4j transaction
        Valid transaction from an active Neo4J session

    query: str
        Query to execute

    Returns
    -------
    dataframe
        Pandas dataframe with results
    """
    try:
        result = tx.run(query)
    except Exception as x:
        raise type(x)(f"tx.run: {query}\n" + str(x))
    if verbose:
        print("Result columns:", result.keys())
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    info = result.consume()
    return df, info

In [ ]:
print("Connect to Neo4J on AI Bench")
aib_neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))

In [ ]:
show_dbs_query = """
SHOW DATABASES WHERE name =~ '.*full-dev'
"""

In [ ]:
# Run the "show databases" statement as a transaction, using default database
try:
    with aib_neo4j_driver.session() as session:
        db_results, info = session.read_transaction(run_query, show_dbs_query)
except Exception as x:
    print("Query:", show_dbs_query)
    raise type(x)(f"read_transaction: {show_dbs_query}\n" + str(x))

print(f"Query complete: {info}")

In [ ]:
# Get the latest available database
NEO4J_DB = db_results.iloc[-1]["name"]
print(f"Using latest PSKG database: {NEO4J_DB}")

In [ ]:
# EUDRAVIGILANCE cases receiving more than one covid vaccine, identify all administered vaccines
sample_query = """
MATCH (c:Case {DataSource:'EUDRAVIGILANCE'})-[:ADMINISTERED]->(v:Vaccine {VaxType:'COVID19'}) 
WITH c,COUNT(v) as vaccinations 
MATCH (c)-[av:ADMINISTERED] -> (v2:Vaccine) 
WHERE vaccinations > 1 RETURN c.CaseId as CaseId, c.ReceivedDate as Received, av.Characterization AS Characterization, v2.GenericName AS GenericName
ORDER BY vaccinations DESC, c.CaseId 
LIMIT 50
"""

In [ ]:
# Run the query as a transaction,
try:
    with aib_neo4j_driver.session(database=NEO4J_DB) as session:
        result, info = session.read_transaction(run_query, sample_query)
except Exception as x:
    print("Query:", sample_query)
    raise type(x)(f"read_transaction: {sample_query}\n" + str(x))

print(f"Query complete: {info}")

In [ ]:
result